In [1]:
from collect_data import *
import torch
from transformers import CLIPProcessor, CLIPModel
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from matplotlib import cm

/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
raw = get_bids_raw(meg_path, patient[0], session[0], '0')

Reading 0 ... 395999  =      0.000 ...   395.999 secs...


/home/matteoc/spect-to-meg/code/collect_data.py:47: RuntimeWarning: The unit for channel(s) MISC 001, MISC 002, MISC 003, MISC 004, MISC 005, MISC 006, MISC 007, MISC 008, MISC 009, MISC 010, MISC 011, MISC 012, MISC 013, MISC 014, MISC 015, MISC 016, MISC 017, MISC 018, MISC 019, MISC 020, MISC 021, MISC 022, MISC 023, MISC 024, MISC 025, MISC 026, MISC 027, MISC 028, MISC 029, MISC 030, MISC 031, MISC 032 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6601 samples (6.601 s)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


In [3]:
get_epochs = get_epochs(raw, task.get("lw1"), float(lw1[0]))

Adding metadata with 18 columns
180 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 180 events and 3201 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)


In [4]:
print('EPOCH_SHAPE: ', get_epochs.get_data().shape)

EPOCH_SHAPE:  (180, 208, 3201)


In [5]:
wav_list_without_numb = list(task.keys())
print(wav_list_without_numb)

['lw1', 'cable_spool_fort', 'easy_money', 'the_black_willow']


In [6]:
audio_path = os.path.join(meg_path, 'collect_data/audio')
audio_list = os.listdir(audio_path)

In [24]:
# CLIP necessità di immagini con 3 canali RGB ? 
colormap = cm.get_cmap('viridis')
audio_data = []
audio_shape = []
for file_name in audio_list:
    file_path = os.path.join(audio_path, file_name)
    tensor = torch.load(file_path)
    audio_shape.append(tensor.shape[0])
    normalized_spectrogram = (tensor - tensor.min()) / (tensor.max() - tensor.min())
    rgba_image = colormap(normalized_spectrogram.numpy())
    rgb_image = torch.tensor(rgba_image[:, :, :, :3]).permute(0, 3, 1, 2)
    audio_data.append(rgb_image)
audio_tensor = list(torch.cat(audio_data, dim=0))
print(len(audio_tensor))

8561


In [ ]:
words_list = []
words_shape = []
for task_name, task_value in task.items():
    print(f"TASK_NAME: {task_name}, TASK_VALUE: {task_value}")
    pieces = []
    if task_name == 'lw1':
        pieces = lw1
    elif task_name == 'cable_spool_fort':
        pieces = cable_spool_fort
    elif task_name == 'easy_money':
        pieces = easy_money
    elif task_name == 'the_black_willow':
        pieces = the_black_willow
    for piece in pieces:
        print(f"PIECE_OF_TASK: {piece}")
        raw = get_bids_raw(meg_path, patient[0], session[0], str(int(task_value)))
        meta = list()
        for annot in raw.annotations:
            d = eval(annot.pop("description"))
            for k, v in annot.items():
                assert k not in d.keys()
                d[k] = v
            meta.append(d)
        meta = pd.DataFrame(meta)
        meta["intercept"] = 1.0
        meta=meta[(meta["kind"]=="word") & (meta["story_uid"]==task_value) & 
                    (meta["sound_id"]==float(piece))]  
        words = meta["word"].to_numpy()
        print('WORDS_SHAPE: ', words.shape)
        words_list.append(words)
        words_shape.append(words.shape[0])

wordres_list = [item for array in words_list for item in array]    

In [30]:
save_words_path = os.path.join(meg_path, 'collect_data/words_audio_encoding.npy')
# np.save(save_words_path, wordres_list)
loaded_list = list(np.load(save_words_path, allow_pickle=True))
print(len(loaded_list))

8561


In [28]:
class SpectrogramTextDataset(Dataset):
    def __init__(self, spectrograms, text_labels):
        self.spectrograms = spectrograms
        self.text_labels = text_labels

    def __len__(self):
        return len(self.spectrograms)

    def __getitem__(self, idx):
        spectrogram = self.spectrograms[idx]
        text = self.text_labels[idx]
        return spectrogram, text

# Tokenizer and model initialization
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16")
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")

dataset = SpectrogramTextDataset(audio_tensor, loaded_list)
dataloader = DataLoader(dataset, shuffle=True)

# Training loop
optimizer = Adam(model.parameters(), lr=1e-4)
